In [2]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00a 0:00:01


In [3]:
%pwd

'/Users/ericdanforth/Downloads'

In [49]:
import PyPDF2
import csv
import pandas as pd

text_list = []
with open('2022_ca_designer_collection_1st_ptg_rev.pdf', 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    for page_number in range(50): #reader.numPages
        text = reader.pages[page_number].extract_text()
        text_list.append([text])

df = pd.DataFrame(text_list, columns=['Text'])
df.to_csv('ca_codes.csv', index=False)

In [50]:
import re
import pandas as pd

# Load the DataFrame from the CSV
# df = pd.read_csv('output.csv')

def process_text(row):
    text = row['Text']
    page_number = None

    if row.name % 2 == 0:  # Even page
        match = re.search(r'(.*?)(\n?)(2022 CALIFORNIA ADMINISTRATIVE CODE )(\d+-\d+)(.*)(Copyright © 2022 ICC)', text, re.DOTALL)
        if match:
            page_number = match.group(4)
            text = match.group(5)
    else:  # Odd page
        match = re.search(r'(.*?)(\n?)(\d+-\d+)( 2022 CALIFORNIA ADMINISTRATIVE CODE)(.*)(Copyright © 2022 ICC)', text, re.DOTALL)
        if match:
            text = match.group(5)
            page_number = match.group(3)

    # Return a new row
    return pd.Series([page_number, text])

# Apply the function to every row in the DataFrame
df[['Page', 'Text']] = df.apply(process_text, axis=1)

# Write the DataFrame back to the CSV
df.to_csv('ca_codes.csv', index=False)
df.head(20)

,Text,Page
0,"TO VIEW, CLICK ON A TITLE.\nHELPFUL LINKS\n202...",None
1,2022 CALIFORNIA \nADMINISTRATIVE CODE\nCALIF...,None
2,IMPORTANT NOTICE\nAct now to keep your code up...,None
3,2022 California Administrative Code\nCaliforn...,None
4,2022 CALIFORNIA ADMINISTRATIVE CODE iiiPREFACE...,None
5,iv 2022 CALIFORNIA ADMINISTRATIVE CODE\nCopyri...,None
6,2022 CALIFORNIA ADMINISTRATIVE CODE vCALIFORNI...,None
7,vi 2022 CALIFORNIA ADMINISTRATIVE CODEHOW TO D...,None
8,2022 CALIFORNIA ADMINISTRATIVE CODE viiTABLE O...,None
9,viii 2022 CALIFORNIA ADMINISTRATIVE CODETABLE ...,None


In [60]:
chap1 = []
for pagenum in range(12,23):
    content = df.iloc[pagenum][0]
    chap1.append(content)
concatenated_string = ''.join(chap1)

In [305]:
concatenated_string

'CHAPTER 1\nADMINISTRATIVE REGULATIONS OF THE \nCALIFORNIA BUILDING STANDARDS COMMISSION\nARTICLE 1\nGENERAL \n1-101. Abbreviations. The following abbreviations shall\napply to Title 24, California Code of Regulations. Abbrevia-\ntions may also be provided in  each of the other 12 parts of\nTitle 24. If an abbreviation in this section conflicts with an\nabbreviation within another part of Title 24, the abbreviation\nreference in the other part shall prevail within that part.\nNote: For information regarding the code provisions adopted\nfor a state agency and the appli cation of such code provisions,\nsee the agency administrative  chapters in the appropriate\nparts of Title 24,  California Code of Regulations. \nAGR Department of Food and Agriculture\nBSC Identifies code provisions by the Building Stan-\ndards Commission\nBSC-CG Identifies the Califo rnia Green Building Stan-\ndards Code (CALGreen) provisions by BSC \nBSCC Identifies code provisio ns by the Board of State\nand Communit

In [463]:
import re

nodes = []
def assign_node():
    if len(nodes) == 0:
        nodes.append(60000000)
        return 60000000
    else:
        node = max(nodes) + 1
        nodes.append(node)
        return node


def parse_text(text, p_node=0, level=0):
#     global result
    my_p_node = str(p_node)
    patterns = [
        r"(CHAPTER \d+)(.*?)(ARTICLE \d.*?)(?=CHAPTER \d+|$)",
        r"(ARTICLE \d+)(.*?)(\d+-\d+\..*?)(?=ARTICLE \d+|$)",
        r"(\d+-\d+\.)(.*?\.)(.*?)(?=\d+-\d+\.|$)",
        r"\n(\([a-z]\)\s)(.*?)(.*?)(?=\n\([a-z]\)\s|$)",
        r"\n(\d+\.\s)(.*?)(.*?)(?=\n\d+\.\s|$)",  
        r"\n([A-Z]\s)(.*?\.)(.*?)(?=[A-Z]\w*|$)",
        r"\n\((\d+\s)\)(.*?\.)(.*?)(?=\(\d+\)|$)",
        r"\n\(([ivxlcdm]+\s)\)(.*?\.)(.*?)(?=\([ivxlcdm]+\)|$)" 
    ]
    result = []

    # Base case: if level is equal to or exceeds the length of patterns, return text in a list
    if level >= len(patterns):
#         return [[text.replace('-\n', '').replace('\n', ' ').strip(), '']]
#         return [my_p_node,assign_node(),text.replace('-\n', '').replace('\n', ' ').strip(),""]
        text_clean = text.replace('-\n', '').replace('\n', ' ').strip()
        return [
            f'"parent_id": ["l_{level}", {my_p_node}]', 
            f'"id": ["l_{level+1}", {assign_node()}]',
            f'"text": "{text_clean}"', 
            f'"title": ""'
        ]

    match_list = re.findall(patterns[level], text, re.DOTALL)

    # If there's no match, go to the next level
    if len(match_list) == 0:
        node = assign_node()
        return parse_text(text, node ,level + 1)
    
    for match in match_list:
        node = assign_node()
        pattern, title, following_text = match[0], match[1], match[2]
#         if level == 3:
#             print(f"pattern: {pattern}, title: {title}, following_text: {following_text[:300]}")
        # Apply the same parsing process for the rest of the text by increasing level
        remaining_parts = parse_text(following_text, node, level + 1 ) #node
#         result.append([my_p_node,node,pattern.replace('-\n', '').replace('\n', ' ').strip(), title.replace('-\n', '').replace('\n', ' ').strip()])
        pattern_clean = pattern.replace('-\n', '').replace('\n', ' ').strip()
        title_clean = title.replace('-\n', '').replace('\n', ' ').strip()

        result.append([
            f'"parent_id": ["l_{level}", {my_p_node}]', 
            f'"id": ["l_{level+1}", {node}]',
            f'"text": "{pattern_clean}"', 
            f'"title": "{title_clean}"'
        ])


        result.append(remaining_parts) #'node:'+str(node),'p_node:'+str(p_node)
    return result



In [464]:
parsed_text = parse_text(concatenated_string)


In [465]:
import json

formatted_dict = json.dumps(parsed_text, indent=4)

print(formatted_dict)

[
    [
        "\"parent_id\": [\"l_0\", 0]",
        "\"id\": [\"l_1\", 60000000]",
        "\"text\": \"CHAPTER 1\"",
        "\"title\": \"ADMINISTRATIVE REGULATIONS OF THE  CALIFORNIA BUILDING STANDARDS COMMISSION\""
    ],
    [
        [
            "\"parent_id\": [\"l_1\", 60000000]",
            "\"id\": [\"l_2\", 60000001]",
            "\"text\": \"ARTICLE 1\"",
            "\"title\": \"GENERAL\""
        ],
        [
            [
                "\"parent_id\": [\"l_2\", 60000001]",
                "\"id\": [\"l_3\", 60000002]",
                "\"text\": \"1-101.\"",
                "\"title\": \"Abbreviations.\""
            ],
            [
                "\"parent_id\": [\"l_8\", 60000007]",
                "\"id\": [\"l_9\", 60000008]",
                "\"text\": \"The following abbreviations shall apply to Title 24, California Code of Regulations. Abbreviations may also be provided in  each of the other 12 parts of Title 24. If an abbreviation in this section conf

In [355]:
import sys

new_limit = 5000  # Set the new recursion depth limit
sys.setrecursionlimit(new_limit)


In [466]:
def flatten_list(nested_list):
    result = []
    for item in nested_list:
        if isinstance(item, list):
            if len(item) > 0 and isinstance(item[0], list):
                result.extend(flatten_list(item))
            else:
                result.append(item)
        else:
            result.append(item)
    return result

In [467]:
print(flatten_list(parsed_text))


[['"parent_id": ["l_0", 0]', '"id": ["l_1", 60000000]', '"text": "CHAPTER 1"', '"title": "ADMINISTRATIVE REGULATIONS OF THE  CALIFORNIA BUILDING STANDARDS COMMISSION"'], ['"parent_id": ["l_1", 60000000]', '"id": ["l_2", 60000001]', '"text": "ARTICLE 1"', '"title": "GENERAL"'], ['"parent_id": ["l_2", 60000001]', '"id": ["l_3", 60000002]', '"text": "1-101."', '"title": "Abbreviations."'], ['"parent_id": ["l_8", 60000007]', '"id": ["l_9", 60000008]', '"text": "The following abbreviations shall apply to Title 24, California Code of Regulations. Abbreviations may also be provided in  each of the other 12 parts of Title 24. If an abbreviation in this section conflicts with an abbreviation within another part of Title 24, the abbreviation reference in the other part shall prevail within that part. Note: For information regarding the code provisions adopted for a state agency and the appli cation of such code provisions, see the agency administrative  chapters in the appropriate parts of Title

In [468]:
print(parsed_text)

[['"parent_id": ["l_0", 0]', '"id": ["l_1", 60000000]', '"text": "CHAPTER 1"', '"title": "ADMINISTRATIVE REGULATIONS OF THE  CALIFORNIA BUILDING STANDARDS COMMISSION"'], [['"parent_id": ["l_1", 60000000]', '"id": ["l_2", 60000001]', '"text": "ARTICLE 1"', '"title": "GENERAL"'], [['"parent_id": ["l_2", 60000001]', '"id": ["l_3", 60000002]', '"text": "1-101."', '"title": "Abbreviations."'], ['"parent_id": ["l_8", 60000007]', '"id": ["l_9", 60000008]', '"text": "The following abbreviations shall apply to Title 24, California Code of Regulations. Abbreviations may also be provided in  each of the other 12 parts of Title 24. If an abbreviation in this section conflicts with an abbreviation within another part of Title 24, the abbreviation reference in the other part shall prevail within that part. Note: For information regarding the code provisions adopted for a state agency and the appli cation of such code provisions, see the agency administrative  chapters in the appropriate parts of Tit